Notebook demonstrating how cluster count covariances are calculated using TJPCov.
* Reads mock sacc file with cluster tracers present
* Calculates cluster count covariances
* Adds cluster count covariances to sacc file

In [22]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pyccl as ccl
import sacc

from tjpcov_new.covariance_clusters import CovarianceClusterCounts

input_yml = './tjpcov_conf_minimal_clusters.yaml'

ModuleNotFoundError: No module named 'tjpcov_new'

In [20]:
sacc_file = sacc.Sacc.load_fits("./mock_clusters.sacc")
min_mass = sacc_file.metadata['min_mass']
N_z = sacc_file.metadata['nbins_cluster_redshift']
N_lambda = sacc_file.metadata['nbins_cluster_richness']

In [ ]:
 # TJPCov covariance

nxn_cov  = CovarianceClusterCounts(input_yml)
cosmo = nxn_cov.get_cosmology()
s = nxn_cov.io.get_sacc_file()
# ell, _ = s.get_ell_cl('cl_00', 'DESgc__0', 'DESgc__0')

cov_ssc = nxn_cov.get_covariance_block(include_b_modes=False)